In [131]:
import wget

In [132]:
# url = "https://raw.githubusercontent.com/alexeygrigorev/minsearch/refs/heads/main/minsearch.py"
# wget.download(url)

In [133]:
#Docoment.json download
# url="https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/refs/heads/main/01-intro/documents.json"
# wget.download(url)

In [134]:
import minsearch

In [135]:
import json

In [136]:
with open('documents.json','rt') as f_in:
    docs_raw=json.load(f_in)

In [ ]:
docs_raw

In [138]:
documents=[]

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course']=course_dict['course']
        documents.append(doc)


In [ ]:
documents

In [ ]:
documents[0]

Index the Document

In [141]:
index =minsearch.Index(
    text_fields=["question","text","section"],
    keyword_fields=["course"]

)

In [142]:
index.fit(documents)


In [143]:
q='the course has already started, can i still enroll?'

In [144]:
boost={'question':3.0,'section':0.5}

results=index.search(
    query=q,
    filter_dict={'course':'data-engineering-zoomcamp'},
    boost_dict=boost,
    num_results=5
)

In [ ]:
results

In [146]:
from openai import OpenAI

In [147]:
from dotenv import load_dotenv
import os
from openai import OpenAI

load_dotenv()

api_key = os.getenv("OPENAI_API_KEY")




In [148]:
client = OpenAI(api_key=api_key)

In [149]:
response=client.chat.completions.create(
    model='gpt-4o',
    messages=[{"role":"user","content":q}]
)

In [150]:
response.choices[0].message.content

"Whether you can still enroll in a course that has already started depends on the specific policies of the institution or organization offering the course. Here are a few steps you can take to find out:\n\n1. **Check the Course Website**: Look for information on enrollment deadlines and late registration policies.\n\n2. **Contact the Instructor**: Reach out to the course instructor to inquire if late enrollment is possible and if there are any prerequisites that you need to meet.\n\n3. **Contact the Admissions Office or Registrar**: These offices can provide information on enrollment policies and might be able to assist in facilitating your late registration.\n\n4. **Consider Online or Self-Paced Options**: If late enrollment in the traditional course isn't possible, see if there are online or self-paced versions of the course available.\n\n5. **Be Prepared for Catching Up**: If you are allowed to enroll late, be ready to catch up on any missed material and assignments to get up to spe

In [151]:
prompt_template="""
You're a course teaching assistance. Answer the QUESTION based on the CONTEXT from the FAQ Databalse. Use only the facts from the CONTEXT when answering the QUESTION.
if the CONTEXT doesn't contain the answer, output NONE

Question :{question}

CONTEXT:
{context}
""".strip()

In [152]:
context = ""
for doc in results:
    context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"

In [ ]:
print(context)

In [154]:
prompt=prompt_template.format(question=q,context=context).strip()

In [156]:
response=client.chat.completions.create(
    model='gpt-4o',
    messages=[{"role":"user","content":prompt}]
)

In [157]:
response.choices[0].message.content

"Yes, you can still enroll after the course has started. Even if you don't register on time, you are eligible to submit the homework. However, be mindful of the deadlines for final projects."